In [1]:
!pip install --quiet openai langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.0 MB/s eta 0:00:00


In [40]:
from PIL import Image
image = Image.open('/content/img1.jpg')
image = image.convert('RGB')
new_image = image.resize((1024, 1024))
new_image.save('inputImage.jpg')

# 1. Vision Analysis

In [41]:
from openai import OpenAI
import base64
import os
from google.colab import userdata

In [42]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [43]:
def encode_image(image_path):
  with open(image_path, "rb") as f:
    return base64.b64encode(f.read()).decode('utf-8')

In [44]:
vision_prompt = """
  You are an extremely knowledgeable Image Analyst.
  Your expertise includes analyzing and understanding every detail of the provided image.

  Task:
  Examine the following image in detail and provide a detailed, factual, and accurate explanation of what the image depicts.
  If this is a well-known location, please provide me with every relevant detail.
  Highlight all the key elements and their significance, and present your analysis in clear, well-structured markdown format.
  Finally, add a list of all the cars, humans, and animals present, including the count and color of the object.
"""

In [45]:
image_path = '/content/inputImage.jpg'
base64_image = encode_image(image_path)

In [46]:
client = OpenAI()
model="gpt-4-vision-preview"
max_tokens=1200

In [47]:
messages = [
    {
        "role" : "user",
        "content":[
            {'type' : 'text',
             "text" : f'{vision_prompt}'},
            {
                "type" : "image_url",
                "image_url" : f"data:image/jpeg;base64,{base64_image}"
            }
        ]
    }
]

In [48]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=max_tokens
)

In [49]:
vision_response = response.choices[0].message.content

In [12]:
vision_response

'### Image Description\n\nThe image is of traffic and parking regulation signs attached to a pole on a city street. There are three signs, each stacked on top of the other, providing detailed instructional information for parking and stopping in the area.\n\n#### Top Sign: No Stopping\n- This is a rectangular red and white sign indicating "NO STOPPING" for specified times:\n  - **7 AM to 9 AM**\n  - **4 PM to 6 PM**\n- Except on Saturdays and Sundays.\n- The small text at the bottom suggests a number to call to recover impounded vehicles, which implies enforcement of the no stopping rule may result in towing.\n\n#### Middle Sign: 2 Hour Parking\n- Below the "NO STOPPING" sign is a rectangular green and white sign that indicates "2 HOUR PARKING":\n  - **9 AM to 4 PM, Monday to Friday**\n  - **8 AM to 6 PM, Monday to Friday**\n  - **8 AM to 8 PM, Saturday**\n- This sign provides a time frame for when 2-hour parking is allowed on weekdays and Saturdays, suggesting a different regulation a

# Conversational Agent

In [50]:
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [51]:
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

In [52]:
from datetime import datetime

def _get_datetime():
    now = datetime.now()
    print(now)
    return now.strftime("%m/%d/%Y, %H:%M:%S")

In [53]:
template = '''
You are a skilled conversational assistant, and the user is asking you questions while sitting in his car.
You are witty, with a sense of creativity.

You know the date and time based on the information provided below.
date: {date}

You also keep track of previous conversations using the chat history.
Chat History: {history}

Your expertise includes understanding each detail of the context provided below.
Context: {response}

Your Task:
Understand the user question given below.

User Question: {input}

Then, respond appropriately to the user's question while giving suggestions based on the context.
While responding, keep your responses short and clear, and use Yes, No, Sure, wherever necessary.
'''

In [54]:
# You are sarcastic, and witty, with a sense of creativity.

In [55]:
prompt = PromptTemplate(
    input_variables=["history","input"],
    template=template,
    partial_variables={ "response": vision_response, "date" : _get_datetime}
)

In [56]:
# prompt.format(input = "Can i park my car here?", history = memory)

In [57]:
llm = ChatOpenAI(model = 'gpt-4', temperature=0.0)

In [58]:
chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

In [59]:
query = "Can i park here?"
response = chain.invoke(query)
response['response']

2024-03-05 21:01:21.245593


"Yes, you can park here. However, please note the restrictions. It's currently Tuesday at 9:01 PM. You can park for two hours between 9 AM and 6 PM from Monday to Friday. But since it's past 6 PM, you're free to park without a time limit. Just remember, the area becomes a passenger loading zone from 8 PM to 3 AM. So, ensure your vehicle doesn't obstruct any loading activities."

In [60]:
type(response['response'])

str

In [22]:
query = "What do you see in the front?"
response = chain.invoke(query)
response['response']

2024-03-05 20:30:27.802622


'The image in front shows three traffic and parking regulation signs on a pole. The top sign indicates "No Stopping" during peak hours on weekdays. The middle sign allows "2 Hour Parking" during specific hours from Monday to Saturday. The bottom sign designates a "Passenger Loading Zone" active from 8 PM to 3 AM. The background suggests an urban setting with a modern building and some trees. Please remember to follow these regulations to avoid fines or towing.'

In [23]:
query = "What animal is that in the front?"
response = chain.invoke(query)
response['response']

2024-03-05 20:30:31.494510


"I'm sorry, but there are no animals present in the image. It's primarily focused on traffic and parking regulation signs. Perhaps you'd like to know more about the signs or the urban setting?"

In [24]:
memory.clear()

In [25]:
query = "Can i turn right?"
response = chain.invoke(query)
response['response']

2024-03-05 20:30:33.668397


"Based on the image description, there's no information about turning restrictions. So, yes, you should be able to turn right. However, please ensure to follow any additional signs or road markings that might be present but not visible in the image. Safe driving!"

In [26]:
# query = "What animal is that in the front?"
# response = chain.invoke(query)
# response['response']

In [27]:
memory.clear()

In [28]:
query = "Can I take the next right with my car?"
response = chain.invoke(query)
response['response']

2024-03-05 20:30:36.946580


'Sure, you can take the next right with your car. However, please be aware of the parking and stopping regulations. If it\'s between 7 AM to 9 AM or 4 PM to 6 PM, you can\'t stop your car due to the "NO STOPPING" rule. Also, if you plan to park, remember the 2-hour parking limit during specified times. Lastly, if it\'s between 8 PM to 3 AM, you can use the area for passenger loading. Drive safe!'

In [29]:
query = "How many black cars do you see?"
response = chain.invoke(query)
response['response']

2024-03-05 20:30:41.045662


"I'm sorry, but there are no cars visible in the image, black or otherwise. However, if you're planning to park your black car here, remember to follow the regulations on the signs to avoid any fines or towing. Drive safe!"

In [30]:
query = "What is happening here?"
response = chain.invoke(query)
response['response']

2024-03-05 20:30:43.451215


'This image showcases traffic and parking regulation signs in a city area. The signs indicate "No Stopping" during peak hours, "2 Hour Parking" during specified times on weekdays and Saturdays, and a "Passenger Loading Zone" active from 8 PM to 3 AM. These signs are crucial for managing traffic flow and providing parking solutions. If you\'re driving in this area, remember to follow these rules to avoid any fines or potential towing.'

In [31]:
query = "Can i park my car here?"
response = chain.invoke(query)
response['response']

2024-03-05 20:30:47.440031


"Sure, you can park your car here, but there are some restrictions to keep in mind. You can park for up to 2 hours from 9 AM to 4 PM on weekdays, and from 8 AM to 6 PM on Saturdays. However, you can't stop your car from 7 AM to 9 AM and 4 PM to 6 PM on weekdays. Also, from 8 PM to 3 AM, this area is reserved for passenger loading. Please follow these rules to avoid any fines or potential towing."

In [32]:
query = "How many traffic signs do you see?"
response = chain.invoke(query)
response['response']

2024-03-05 20:30:52.384162


'There are three traffic signs visible in the image. They provide important information about stopping, parking, and passenger loading regulations in this area. Please remember to follow these rules to ensure smooth traffic flow and avoid any potential fines or towing.'

In [33]:
# query = "When can i park my car here today?"
# response = chain.invoke(query)
# response['response']

In [34]:
# query = "What did i previously ask?"
# response = chain.invoke(query)
# response['response']

In [35]:
# query = "Where was this picture taken?"
# response = chain.run(query)
# response

In [36]:
# memory.clear()

In [37]:
# query = "can i park here?"
# response = chain.invoke(query)
# response['response']

In [38]:
query = "can i over take?"
response = chain.invoke(query)
response['response']

2024-03-05 20:30:54.636353


"Sure, you can overtake, but please remember to do so safely and responsibly. Always check your mirrors and blind spots, signal your intentions, and ensure there's enough space to complete the maneuver without disrupting other road users. However, the image doesn't provide any specific overtaking restrictions, so local traffic laws should be followed. Drive safe!"